In [1]:
%load_ext autoreload
%autoreload 2

# Get parent directory and add to sys.path
import os
import sys

parent_dir = os.path.dirname(os.getcwd())
sys.path.append(parent_dir)

# Require ipympl
%matplotlib widget 

In [2]:
import numpy as np
from src.rocket import Rocket
from src.vel_rocket_vis import RocketVis

rocket_obj_path = os.path.join(parent_dir, "Cartoon_rocket.obj")
rocket_params_path = os.path.join(parent_dir, "rocket.yaml")

In [3]:
Ts = 0.05

rocket = Rocket(Ts=Ts, model_params_filepath=rocket_params_path)
u = np.array([0.0, 0.1, 60.0, 0.0])  # (Assign appropriately)
b_F, b_M = rocket.getForceAndMomentFromThrust(u)

x = np.array(
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]
)  # (Assign appropriately)
x_dot, _ = rocket.f(x, u)
print(x_dot)

[ 0.         -5.88291971  0.          0.          0.          0.
  0.88106984  0.         -1.02469024  0.          0.          0.        ]


In [4]:
Tf = 7
x0 = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])  # initial state
u = np.array([0.0, 0.0, 66.66666667, 0.0])  # initial input

rocket = Rocket(Ts=Ts, model_params_filepath=rocket_params_path)

t_cl, x_cl, u_cl = rocket.simulate(x0, Tf, u, method="nonlinear")

vis = RocketVis(rocket, rocket_obj_path)
vis.anim_rate = 1.0
vis.animate(t_cl, x_cl, u_cl)

Simulating time 0.00:
Simulating time 0.05:
Simulating time 0.10:
Simulating time 0.15:
Simulating time 0.20:
Simulating time 0.25:
Simulating time 0.30:
Simulating time 0.35:
Simulating time 0.40:
Simulating time 0.45:
Simulating time 0.50:
Simulating time 0.55:
Simulating time 0.60:
Simulating time 0.65:
Simulating time 0.70:
Simulating time 0.75:
Simulating time 0.80:
Simulating time 0.85:
Simulating time 0.90:
Simulating time 0.95:
Simulating time 1.00:
Simulating time 1.05:
Simulating time 1.10:
Simulating time 1.15:
Simulating time 1.20:
Simulating time 1.25:
Simulating time 1.30:
Simulating time 1.35:
Simulating time 1.40:
Simulating time 1.45:
Simulating time 1.50:
Simulating time 1.55:
Simulating time 1.60:
Simulating time 1.65:
Simulating time 1.70:
Simulating time 1.75:
Simulating time 1.80:
Simulating time 1.85:
Simulating time 1.90:
Simulating time 1.95:
Simulating time 2.00:
Simulating time 2.05:
Simulating time 2.10:
Simulating time 2.15:
Simulating time 2.20:
Simulating

AppLayout(children=(HBox(children=(Play(value=0, description='Press play', max=139, step=2), IntSlider(value=0…

{'fig': <Figure size 640x480 with 16 Axes>,
 'axes': [<Axes: ylabel='inputs'>,
  <Axes: >,
  <Axes: >,
  <Axes: >,
  <Axes: title={'center': 'Subsystem Y'}>,
  <Axes: title={'center': 'Subsystem X'}, ylabel='$\\omega_{\\alpha\\beta\\gamma}$ (deg/s)'>,
  <Axes: title={'center': 'Subsystem Roll'}>,
  <Axes: >,
  <Axes: ylabel='$\\alpha\\beta\\gamma$ (deg)'>,
  <Axes: >,
  <Axes: ylabel='$v$ (m/s)'>,
  <Axes: >,
  <Axes: title={'center': 'Subsystem Z'}>,
  <Axes: ylabel='$\\text{pos}$ (m)'>,
  <Axes: >,
  <Axes: >],
 'plotter': <pyvista.plotting.plotter.Plotter at 0x2c48596b8f0>,
 'scene_objects': {'rocket_actor': Actor (0x2c48589dc00)
    Center:                     (0.32006999999999997, -0.0015085000000000237, 0.5884844999999999)
    Pickable:                   True
    Position:                   (0.0, 0.0, 0.0)
    Scale:                      (1.0, 1.0, 1.0)
    Visible:                    True
    X Bounds                    -6.402E-01, 1.280E+00
    Y Bounds                    -1.21

# Todo 1.2: Test Rocket Dynamics

In this section, we'll test various inputs to understand how the rocket behaves:

**State vector:** `x = [wx, wy, wz, alpha, beta, gamma, vx, vy, vz, x, y, z]`

**Input vector:** `u = [d1, d2, Pavg, Pdiff]`
- `d1, d2`: servo deflections (rad), range: ±15° = ±0.262 rad
- `Pavg`: average throttle (%), range: [10, 90] (we use [40, 80] for safety)
- `Pdiff`: differential throttle (%), range: [-20, 20]

## Tests to perform:
1. **Hover** - Find equilibrium
2. **Ascend/Descend** - Vertical motion without tipping
3. **Rotate** - About body x, y, z axes
4. **Fly** - Along world x, y, z axes

In [5]:
# TEST 1: HOVER - Find equilibrium throttle
# For hover, thrust must balance gravity
# The trim() function finds the steady-state equilibrium

xs, us = rocket.trim()
print(f"Equilibrium (trim) state: xs = {xs}")
print(f"Equilibrium (trim) input: us = {us}")
print(f"\nHover throttle: Pavg = {us[2]:.2f}%")

# Test hover
u_hover = np.array([0.0, 0.0, us[2], 0.0])
x0 = np.zeros(12)
Tf = 7.0

print(f"\nSimulating hover for {Tf} seconds...")
T, X, U = rocket.simulate(x0, Tf, u_hover, method='nonlinear')
print(f"Final position: z = {X[11, -1]:.3f}m (should be ≈0 for perfect hover)")
print(f"Final velocity: vz = {X[8, -1]:.3f}m/s (should be ≈0)")

# Visualize
vis.animate(T, X, U)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

Equilibrium (trim) state: xs = [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Equilibrium (trim) input: us = [ 0.          0.         66.66666667  0.        ]

Hover throttle: Pavg = 66.67%

Simulating hover for 7.0 seconds...
Simulating time 0.00:
Simulating time 0.05:
Simulating time 0.10:
Simulating time 0.15:
Simulating time 0.20:
Simulating time 0.25:
Simulating time 0.30:
Simulating time 0.35:
Simulating time 0.40:
Simulating time 0.45:
Simulating time 0.50:
Simulating time 0.55:
Simulating time 0.60:
Simulating time 0.65:
Simulating time 0.70:
Simulating time 0.75:
Simulating time 0.80:
Simulating time 0.85:

AppLayout(children=(HBox(children=(Play(value=0, description='Press play', max=139, step=2), IntSlider(value=0…

{'fig': <Figure size 640x480 with 16 Axes>,
 'axes': [<Axes: ylabel='inputs'>,
  <Axes: >,
  <Axes: >,
  <Axes: >,
  <Axes: title={'center': 'Subsystem Y'}>,
  <Axes: title={'center': 'Subsystem X'}, ylabel='$\\omega_{\\alpha\\beta\\gamma}$ (deg/s)'>,
  <Axes: title={'center': 'Subsystem Roll'}>,
  <Axes: >,
  <Axes: ylabel='$\\alpha\\beta\\gamma$ (deg)'>,
  <Axes: >,
  <Axes: ylabel='$v$ (m/s)'>,
  <Axes: >,
  <Axes: title={'center': 'Subsystem Z'}>,
  <Axes: ylabel='$\\text{pos}$ (m)'>,
  <Axes: >,
  <Axes: >],
 'plotter': <pyvista.plotting.plotter.Plotter at 0x2c4b43e4890>,
 'scene_objects': {'rocket_actor': Actor (0x2c4b44c0ca0)
    Center:                     (0.32006999999999997, -0.0015085000000000237, 0.5884844999999999)
    Pickable:                   True
    Position:                   (0.0, 0.0, 0.0)
    Scale:                      (1.0, 1.0, 1.0)
    Visible:                    True
    X Bounds                    -6.402E-01, 1.280E+00
    Y Bounds                    -1.21

In [6]:
# TEST 2a: ASCEND VERTICALLY
# Strategy: Keep d1=0, d2=0 (no tilting), increase Pavg above hover throttle

u_ascend = np.array([0.0, 0.0, 75.0, 0.0])  # Higher throttle
x0 = np.zeros(12)
Tf = 7.0

print("Testing ASCEND with higher throttle...")
print(f"Input: u = {u_ascend}")
T, X, U = rocket.simulate(x0, Tf, u_ascend, method='nonlinear')

print(f"\nResults after {Tf}s:")
print(f"  Final height: z = {X[11, -1]:.2f}m")
print(f"  Final velocity: vz = {X[8, -1]:.2f}m/s")
print(f"  Final angles (deg): alpha={np.rad2deg(X[3,-1]):.2f}, beta={np.rad2deg(X[4,-1]):.2f}, gamma={np.rad2deg(X[5,-1]):.2f}")
print(f"  (Angles should be close to 0 - no tipping)")

# Visualize
vis.animate(T, X, U)

Testing ASCEND with higher throttle...
Input: u = [ 0.  0. 75.  0.]
Simulating time 0.00:
Simulating time 0.05:
Simulating time 0.10:
Simulating time 0.15:
Simulating time 0.20:
Simulating time 0.25:
Simulating time 0.30:
Simulating time 0.35:
Simulating time 0.40:
Simulating time 0.45:
Simulating time 0.50:
Simulating time 0.55:
Simulating time 0.60:
Simulating time 0.65:
Simulating time 0.70:
Simulating time 0.75:
Simulating time 0.80:
Simulating time 0.85:
Simulating time 0.90:
Simulating time 0.95:
Simulating time 1.00:
Simulating time 1.05:
Simulating time 1.10:
Simulating time 1.15:
Simulating time 1.20:
Simulating time 1.25:
Simulating time 1.30:
Simulating time 1.35:
Simulating time 1.40:
Simulating time 1.45:
Simulating time 1.50:
Simulating time 1.55:
Simulating time 1.60:
Simulating time 1.65:
Simulating time 1.70:
Simulating time 1.75:
Simulating time 1.80:
Simulating time 1.85:
Simulating time 1.90:
Simulating time 1.95:
Simulating time 2.00:
Simulating time 2.05:
Simulati

AppLayout(children=(HBox(children=(Play(value=0, description='Press play', max=139, step=2), IntSlider(value=0…

{'fig': <Figure size 640x480 with 16 Axes>,
 'axes': [<Axes: ylabel='inputs'>,
  <Axes: >,
  <Axes: >,
  <Axes: >,
  <Axes: title={'center': 'Subsystem Y'}>,
  <Axes: title={'center': 'Subsystem X'}, ylabel='$\\omega_{\\alpha\\beta\\gamma}$ (deg/s)'>,
  <Axes: title={'center': 'Subsystem Roll'}>,
  <Axes: >,
  <Axes: ylabel='$\\alpha\\beta\\gamma$ (deg)'>,
  <Axes: >,
  <Axes: ylabel='$v$ (m/s)'>,
  <Axes: >,
  <Axes: title={'center': 'Subsystem Z'}>,
  <Axes: ylabel='$\\text{pos}$ (m)'>,
  <Axes: >,
  <Axes: >],
 'plotter': <pyvista.plotting.plotter.Plotter at 0x2c4b440e120>,
 'scene_objects': {'rocket_actor': Actor (0x2c4b48bdd20)
    Center:                     (0.32006999999999997, -0.0015085000000000237, 0.5884844999999999)
    Pickable:                   True
    Position:                   (0.0, 0.0, 0.0)
    Scale:                      (1.0, 1.0, 1.0)
    Visible:                    True
    X Bounds                    -6.402E-01, 1.280E+00
    Y Bounds                    -1.21

In [7]:
# TEST 2b: DESCEND VERTICALLY
# Strategy: Keep d1=0, d2=0, decrease Pavg below hover throttle (but above 40%)

u_descend = np.array([0.0, 0.0, 50.0, 0.0])  # Lower throttle
x0 = np.zeros(12)
x0[11] = 10.0  # Start at height 10m
Tf = 7.0

print("Testing DESCEND with lower throttle...")
print(f"Input: u = {u_descend}")
print(f"Starting from height: z0 = {x0[11]}m")
T, X, U = rocket.simulate(x0, Tf, u_descend, method='nonlinear')

print(f"\nResults after {Tf}s:")
print(f"  Final height: z = {X[11, -1]:.2f}m (started at 10m)")
print(f"  Final velocity: vz = {X[8, -1]:.2f}m/s (negative = descending)")
print(f"  Final angles (deg): alpha={np.rad2deg(X[3,-1]):.2f}, beta={np.rad2deg(X[4,-1]):.2f}, gamma={np.rad2deg(X[5,-1]):.2f}")

# Visualize
vis.animate(T, X, U)

Testing DESCEND with lower throttle...
Input: u = [ 0.  0. 50.  0.]
Starting from height: z0 = 10.0m
Simulating time 0.00:
Simulating time 0.05:
Simulating time 0.10:
Simulating time 0.15:
Simulating time 0.20:
Simulating time 0.25:
Simulating time 0.30:
Simulating time 0.35:
Simulating time 0.40:
Simulating time 0.45:
Simulating time 0.50:
Simulating time 0.55:
Simulating time 0.60:
Simulating time 0.65:
Simulating time 0.70:
Simulating time 0.75:
Simulating time 0.80:
Simulating time 0.85:
Simulating time 0.90:
Simulating time 0.95:
Simulating time 1.00:
Simulating time 1.05:
Simulating time 1.10:
Simulating time 1.15:
Simulating time 1.20:
Simulating time 1.25:
Simulating time 1.30:
Simulating time 1.35:
Simulating time 1.40:
Simulating time 1.45:
Simulating time 1.50:
Simulating time 1.55:
Simulating time 1.60:
Simulating time 1.65:
Simulating time 1.70:
Simulating time 1.75:
Simulating time 1.80:
Simulating time 1.85:
Simulating time 1.90:
Simulating time 1.95:
Simulating time 2.0

AppLayout(children=(HBox(children=(Play(value=0, description='Press play', max=139, step=2), IntSlider(value=0…

{'fig': <Figure size 640x480 with 16 Axes>,
 'axes': [<Axes: ylabel='inputs'>,
  <Axes: >,
  <Axes: >,
  <Axes: >,
  <Axes: title={'center': 'Subsystem Y'}>,
  <Axes: title={'center': 'Subsystem X'}, ylabel='$\\omega_{\\alpha\\beta\\gamma}$ (deg/s)'>,
  <Axes: title={'center': 'Subsystem Roll'}>,
  <Axes: >,
  <Axes: ylabel='$\\alpha\\beta\\gamma$ (deg)'>,
  <Axes: >,
  <Axes: ylabel='$v$ (m/s)'>,
  <Axes: >,
  <Axes: title={'center': 'Subsystem Z'}>,
  <Axes: ylabel='$\\text{pos}$ (m)'>,
  <Axes: >,
  <Axes: >],
 'plotter': <pyvista.plotting.plotter.Plotter at 0x2c4b5248980>,
 'scene_objects': {'rocket_actor': Actor (0x2c48b066c20)
    Center:                     (0.32006999999999997, -0.0015085000000000237, 10.5884845)
    Pickable:                   True
    Position:                   (0.0, 0.0, 0.0)
    Scale:                      (1.0, 1.0, 1.0)
    Visible:                    True
    X Bounds                    -6.402E-01, 1.280E+00
    Y Bounds                    -1.212E+00, 1

In [8]:
# TEST 3a: ROTATE ABOUT BODY X-AXIS
# Strategy: Use d1 (servo 1) to create moment about x-axis
# From PDF: d1 deflects servo about xb axis

u_rotate_x = np.array([0.15, 0.0, us[2], 0.0])  # d1 ≈ 8.6°, hover throttle
x0 = np.zeros(12)
Tf = 5.0

print("Testing ROTATION about BODY X-AXIS...")
print(f"Input: u = {u_rotate_x}")
print(f"  d1 = {np.rad2deg(u_rotate_x[0]):.1f}° (creates moment about xb)")
T, X, U = rocket.simulate(x0, Tf, u_rotate_x, method='nonlinear')

print(f"\nResults after {Tf}s:")
print(f"  Angular velocity: wx = {X[0, -1]:.3f} rad/s")
print(f"  Euler angle: alpha = {np.rad2deg(X[3, -1]):.2f}°")
print(f"  Position drift: x={X[9,-1]:.2f}m, y={X[10,-1]:.2f}m, z={X[11,-1]:.2f}m")

# Visualize
vis.animate(T, X, U)

Testing ROTATION about BODY X-AXIS...
Input: u = [ 0.15        0.         66.66666667  0.        ]
  d1 = 8.6° (creates moment about xb)
Simulating time 0.00:
Simulating time 0.05:
Simulating time 0.10:
Simulating time 0.15:
Simulating time 0.20:
Simulating time 0.25:
Simulating time 0.30:
Simulating time 0.35:
Simulating time 0.40:
Simulating time 0.45:
Simulating time 0.50:
Simulating time 0.55:
Simulating time 0.60:
Simulating time 0.65:
Simulating time 0.70:
Simulating time 0.75:
Simulating time 0.80:
Simulating time 0.85:
Simulating time 0.90:
Simulating time 0.95:
Simulating time 1.00:
Simulating time 1.05:
Simulating time 1.10:
Simulating time 1.15:
Simulating time 1.20:
Simulating time 1.25:
Simulating time 1.30:
Simulating time 1.35:
Simulating time 1.40:
Simulating time 1.45:
Simulating time 1.50:
Simulating time 1.55:
Simulating time 1.60:
Simulating time 1.65:
Simulating time 1.70:
Simulating time 1.75:
Simulating time 1.80:
Simulating time 1.85:
Simulating time 1.90:
Simul

AppLayout(children=(HBox(children=(Play(value=0, description='Press play', max=99, step=2), IntSlider(value=0,…

{'fig': <Figure size 640x480 with 16 Axes>,
 'axes': [<Axes: ylabel='inputs'>,
  <Axes: >,
  <Axes: >,
  <Axes: >,
  <Axes: title={'center': 'Subsystem Y'}>,
  <Axes: title={'center': 'Subsystem X'}, ylabel='$\\omega_{\\alpha\\beta\\gamma}$ (deg/s)'>,
  <Axes: title={'center': 'Subsystem Roll'}>,
  <Axes: >,
  <Axes: ylabel='$\\alpha\\beta\\gamma$ (deg)'>,
  <Axes: >,
  <Axes: ylabel='$v$ (m/s)'>,
  <Axes: >,
  <Axes: title={'center': 'Subsystem Z'}>,
  <Axes: ylabel='$\\text{pos}$ (m)'>,
  <Axes: >,
  <Axes: >],
 'plotter': <pyvista.plotting.plotter.Plotter at 0x2c4b4f21910>,
 'scene_objects': {'rocket_actor': Actor (0x2c4b4febd00)
    Center:                     (0.32006999999999997, -0.0015085000000000237, 0.5884844999999999)
    Pickable:                   True
    Position:                   (0.0, 0.0, 0.0)
    Scale:                      (1.0, 1.0, 1.0)
    Visible:                    True
    X Bounds                    -6.402E-01, 1.280E+00
    Y Bounds                    -1.21

In [9]:
# TEST 3b: ROTATE ABOUT BODY Y-AXIS
# Strategy: Use d2 (servo 2) to create moment about y-axis
# From PDF: d2 deflects servo about rotated yb axis

u_rotate_y = np.array([0.0, 0.15, us[2], 0.0])  # d2 ≈ 8.6°, hover throttle
x0 = np.zeros(12)
Tf = 5.0

print("Testing ROTATION about BODY Y-AXIS...")
print(f"Input: u = {u_rotate_y}")
print(f"  d2 = {np.rad2deg(u_rotate_y[1]):.1f}° (creates moment about yb)")
T, X, U = rocket.simulate(x0, Tf, u_rotate_y, method='nonlinear')

print(f"\nResults after {Tf}s:")
print(f"  Angular velocity: wy = {X[1, -1]:.3f} rad/s")
print(f"  Euler angle: beta = {np.rad2deg(X[4, -1]):.2f}°")
print(f"  Position drift: x={X[9,-1]:.2f}m, y={X[10,-1]:.2f}m, z={X[11,-1]:.2f}m")

# Visualize
vis.animate(T, X, U)

Testing ROTATION about BODY Y-AXIS...
Input: u = [ 0.          0.15       66.66666667  0.        ]
  d2 = 8.6° (creates moment about yb)
Simulating time 0.00:
Simulating time 0.05:
Simulating time 0.10:
Simulating time 0.15:
Simulating time 0.20:
Simulating time 0.25:
Simulating time 0.30:
Simulating time 0.35:
Simulating time 0.40:
Simulating time 0.45:
Simulating time 0.50:
Simulating time 0.55:
Simulating time 0.60:
Simulating time 0.65:
Simulating time 0.70:
Simulating time 0.75:
Simulating time 0.80:
Simulating time 0.85:
Simulating time 0.90:
Simulating time 0.95:
Simulating time 1.00:
Simulating time 1.05:
Simulating time 1.10:
Simulating time 1.15:
Simulating time 1.20:
Simulating time 1.25:
Simulating time 1.30:
Simulating time 1.35:
Simulating time 1.40:
Simulating time 1.45:
Simulating time 1.50:
Simulating time 1.55:
Simulating time 1.60:
Simulating time 1.65:
Simulating time 1.70:
Simulating time 1.75:
Simulating time 1.80:
Simulating time 1.85:
Simulating time 1.90:
Simul

AppLayout(children=(HBox(children=(Play(value=0, description='Press play', max=99, step=2), IntSlider(value=0,…

{'fig': <Figure size 640x480 with 16 Axes>,
 'axes': [<Axes: ylabel='inputs'>,
  <Axes: >,
  <Axes: >,
  <Axes: >,
  <Axes: title={'center': 'Subsystem Y'}>,
  <Axes: title={'center': 'Subsystem X'}, ylabel='$\\omega_{\\alpha\\beta\\gamma}$ (deg/s)'>,
  <Axes: title={'center': 'Subsystem Roll'}>,
  <Axes: >,
  <Axes: ylabel='$\\alpha\\beta\\gamma$ (deg)'>,
  <Axes: >,
  <Axes: ylabel='$v$ (m/s)'>,
  <Axes: >,
  <Axes: title={'center': 'Subsystem Z'}>,
  <Axes: ylabel='$\\text{pos}$ (m)'>,
  <Axes: >,
  <Axes: >],
 'plotter': <pyvista.plotting.plotter.Plotter at 0x2c4b6b3f830>,
 'scene_objects': {'rocket_actor': Actor (0x2c4b86cc3a0)
    Center:                     (0.32006999999999997, -0.0015085000000000237, 0.5884844999999999)
    Pickable:                   True
    Position:                   (0.0, 0.0, 0.0)
    Scale:                      (1.0, 1.0, 1.0)
    Visible:                    True
    X Bounds                    -6.402E-01, 1.280E+00
    Y Bounds                    -1.21

In [10]:
# TEST 3c: ROTATE ABOUT BODY Z-AXIS (YAW)
# Strategy: Use Pdiff (differential throttle) to create yaw moment
# When motors spin at different speeds, creates torque about zb

u_rotate_z = np.array([0.0, 0.0, us[2], 10.0])  # Pdiff = 10%, hover throttle
x0 = np.zeros(12)
Tf = 5.0

print("Testing ROTATION about BODY Z-AXIS (YAW)...")
print(f"Input: u = {u_rotate_z}")
print(f"  Pdiff = {u_rotate_z[3]:.1f}% (creates torque differential)")
T, X, U = rocket.simulate(x0, Tf, u_rotate_z, method='nonlinear')

print(f"\nResults after {Tf}s:")
print(f"  Angular velocity: wz = {X[2, -1]:.3f} rad/s")
print(f"  Euler angle: gamma = {np.rad2deg(X[5, -1]):.2f}°")
print(f"  Position: x={X[9,-1]:.2f}m, y={X[10,-1]:.2f}m, z={X[11,-1]:.2f}m")

# Visualize
vis.animate(T, X, U)

Testing ROTATION about BODY Z-AXIS (YAW)...
Input: u = [ 0.          0.         66.66666667 10.        ]
  Pdiff = 10.0% (creates torque differential)
Simulating time 0.00:
Simulating time 0.05:
Simulating time 0.10:
Simulating time 0.15:
Simulating time 0.20:
Simulating time 0.25:
Simulating time 0.30:
Simulating time 0.35:
Simulating time 0.40:
Simulating time 0.45:
Simulating time 0.50:
Simulating time 0.55:
Simulating time 0.60:
Simulating time 0.65:
Simulating time 0.70:
Simulating time 0.75:
Simulating time 0.80:
Simulating time 0.85:
Simulating time 0.90:
Simulating time 0.95:
Simulating time 1.00:
Simulating time 1.05:
Simulating time 1.10:
Simulating time 1.15:
Simulating time 1.20:
Simulating time 1.25:
Simulating time 1.30:
Simulating time 1.35:
Simulating time 1.40:
Simulating time 1.45:
Simulating time 1.50:
Simulating time 1.55:
Simulating time 1.60:
Simulating time 1.65:
Simulating time 1.70:
Simulating time 1.75:
Simulating time 1.80:
Simulating time 1.85:
Simulating ti

AppLayout(children=(HBox(children=(Play(value=0, description='Press play', max=99, step=2), IntSlider(value=0,…

{'fig': <Figure size 640x480 with 16 Axes>,
 'axes': [<Axes: ylabel='inputs'>,
  <Axes: >,
  <Axes: >,
  <Axes: >,
  <Axes: title={'center': 'Subsystem Y'}>,
  <Axes: title={'center': 'Subsystem X'}, ylabel='$\\omega_{\\alpha\\beta\\gamma}$ (deg/s)'>,
  <Axes: title={'center': 'Subsystem Roll'}>,
  <Axes: >,
  <Axes: ylabel='$\\alpha\\beta\\gamma$ (deg)'>,
  <Axes: >,
  <Axes: ylabel='$v$ (m/s)'>,
  <Axes: >,
  <Axes: title={'center': 'Subsystem Z'}>,
  <Axes: ylabel='$\\text{pos}$ (m)'>,
  <Axes: >,
  <Axes: >],
 'plotter': <pyvista.plotting.plotter.Plotter at 0x2c55a0faed0>,
 'scene_objects': {'rocket_actor': Actor (0x2c55aae6e00)
    Center:                     (0.32006999999999997, -0.0015085000000000237, 0.5884844999999999)
    Pickable:                   True
    Position:                   (0.0, 0.0, 0.0)
    Scale:                      (1.0, 1.0, 1.0)
    Visible:                    True
    X Bounds                    -6.402E-01, 1.280E+00
    Y Bounds                    -1.21

In [18]:
# TEST 4a: FLY ALONG WORLD X-AXIS
# Strategy: Tilt rocket with d2 to redirect thrust in x direction
# NOTE: From observations, when d2 > 0, thrust actually points in -x direction!
# This is due to the body frame orientation (see rocket diagram)

u_fly_x = np.array([0.0, -0.10, 70.0, 0.0])  # d2 < 0 for +x motion (inverted!)
x0 = np.zeros(12)
Tf = 7.0

print("Testing FLY ALONG WORLD X-AXIS...")
print(f"Input: u = {u_fly_x}")
print(f"  d2 = {np.rad2deg(u_fly_x[1]):.1f}° (negative d2 for +x motion)")
print(f"  NOTE: Sign is opposite to naive expectation due to body frame orientation")
T, X, U = rocket.simulate(x0, Tf, u_fly_x, method='nonlinear')

print(f"\nResults after {Tf}s:")
print(f"  Final position: x={X[9,-1]:.2f}m, y={X[10,-1]:.2f}m, z={X[11,-1]:.2f}m")
print(f"  Final velocity: vx={X[6,-1]:.2f}m/s, vy={X[7,-1]:.2f}m/s, vz={X[8,-1]:.2f}m/s")
print(f"  Main motion should be in +x direction")

# Visualize
vis.animate(T, X, U)

Testing FLY ALONG WORLD X-AXIS...
Input: u = [ 0.  -0.1 70.   0. ]
  d2 = -5.7° (negative d2 for +x motion)
  NOTE: Sign is opposite to naive expectation due to body frame orientation
Simulating time 0.00:
Simulating time 0.05:
Simulating time 0.10:
Simulating time 0.15:
Simulating time 0.20:
Simulating time 0.25:
Simulating time 0.30:
Simulating time 0.35:
Simulating time 0.40:
Simulating time 0.45:
Simulating time 0.50:
Simulating time 0.55:
Simulating time 0.60:
Simulating time 0.65:
Simulating time 0.70:
Simulating time 0.75:
Simulating time 0.80:
Simulating time 0.85:
Simulating time 0.90:
Simulating time 0.95:
Simulating time 1.00:
Simulating time 1.05:
Simulating time 1.10:
Simulating time 1.15:
Simulating time 1.20:
Simulating time 1.25:
Simulating time 1.30:
Simulating time 1.35:
Simulating time 1.40:
Simulating time 1.45:
Simulating time 1.50:
Simulating time 1.55:
Simulating time 1.60:
Simulating time 1.65:
Simulating time 1.70:
Simulating time 1.75:
Simulating time 1.80:
Si

AppLayout(children=(HBox(children=(Play(value=0, description='Press play', max=139, step=2), IntSlider(value=0…

{'fig': <Figure size 640x480 with 16 Axes>,
 'axes': [<Axes: ylabel='inputs'>,
  <Axes: >,
  <Axes: >,
  <Axes: >,
  <Axes: title={'center': 'Subsystem Y'}>,
  <Axes: title={'center': 'Subsystem X'}, ylabel='$\\omega_{\\alpha\\beta\\gamma}$ (deg/s)'>,
  <Axes: title={'center': 'Subsystem Roll'}>,
  <Axes: >,
  <Axes: ylabel='$\\alpha\\beta\\gamma$ (deg)'>,
  <Axes: >,
  <Axes: ylabel='$v$ (m/s)'>,
  <Axes: >,
  <Axes: title={'center': 'Subsystem Z'}>,
  <Axes: ylabel='$\\text{pos}$ (m)'>,
  <Axes: >,
  <Axes: >],
 'plotter': <pyvista.plotting.plotter.Plotter at 0x2c489b2b2c0>,
 'scene_objects': {'rocket_actor': Actor (0x2c489b88460)
    Center:                     (0.32006999999999997, -0.0015085000000000237, 0.5884844999999999)
    Pickable:                   True
    Position:                   (0.0, 0.0, 0.0)
    Scale:                      (1.0, 1.0, 1.0)
    Visible:                    True
    X Bounds                    -6.402E-01, 1.280E+00
    Y Bounds                    -1.21

In [17]:
# TEST 4b: FLY ALONG WORLD Y-AXIS
# Strategy: Tilt rocket with d1 to redirect thrust in y direction
# CORRECTION: Despite equation (1), empirically d1 > 0 gives +y motion!

u_fly_y = np.array([0.10, 0.0, 70.0, 0.0])  # d1 > 0 for +y motion (empirically determined)
x0 = np.zeros(12)
Tf = 7.0

print("Testing FLY ALONG WORLD Y-AXIS...")
print(f"Input: u = {u_fly_y}")
print(f"  d1 = {np.rad2deg(u_fly_y[0]):.1f}° (positive d1 for +y motion)")
print(f"  NOTE: Sign behavior differs from naive interpretation of eq(1)")
T, X, U = rocket.simulate(x0, Tf, u_fly_y, method='nonlinear')

print(f"\nResults after {Tf}s:")
print(f"  Final position: x={X[9,-1]:.2f}m, y={X[10,-1]:.2f}m, z={X[11,-1]:.2f}m")
print(f"  Final velocity: vx={X[6,-1]:.2f}m/s, vy={X[7,-1]:.2f}m/s, vz={X[8,-1]:.2f}m/s")
print(f"  Main motion should be in +y direction")

# Visualize
vis.animate(T, X, U)

Testing FLY ALONG WORLD Y-AXIS...
Input: u = [ 0.1  0.  70.   0. ]
  d1 = 5.7° (positive d1 for +y motion)
  NOTE: Sign behavior differs from naive interpretation of eq(1)
Simulating time 0.00:
Simulating time 0.05:
Simulating time 0.10:
Simulating time 0.15:
Simulating time 0.20:
Simulating time 0.25:
Simulating time 0.30:
Simulating time 0.35:
Simulating time 0.40:
Simulating time 0.45:
Simulating time 0.50:
Simulating time 0.55:
Simulating time 0.60:
Simulating time 0.65:
Simulating time 0.70:
Simulating time 0.75:
Simulating time 0.80:
Simulating time 0.85:
Simulating time 0.90:
Simulating time 0.95:
Simulating time 1.00:
Simulating time 1.05:
Simulating time 1.10:
Simulating time 1.15:
Simulating time 1.20:
Simulating time 1.25:
Simulating time 1.30:
Simulating time 1.35:
Simulating time 1.40:
Simulating time 1.45:
Simulating time 1.50:
Simulating time 1.55:
Simulating time 1.60:
Simulating time 1.65:
Simulating time 1.70:
Simulating time 1.75:
Simulating time 1.80:
Simulating tim

AppLayout(children=(HBox(children=(Play(value=0, description='Press play', max=139, step=2), IntSlider(value=0…

{'fig': <Figure size 640x480 with 16 Axes>,
 'axes': [<Axes: ylabel='inputs'>,
  <Axes: >,
  <Axes: >,
  <Axes: >,
  <Axes: title={'center': 'Subsystem Y'}>,
  <Axes: title={'center': 'Subsystem X'}, ylabel='$\\omega_{\\alpha\\beta\\gamma}$ (deg/s)'>,
  <Axes: title={'center': 'Subsystem Roll'}>,
  <Axes: >,
  <Axes: ylabel='$\\alpha\\beta\\gamma$ (deg)'>,
  <Axes: >,
  <Axes: ylabel='$v$ (m/s)'>,
  <Axes: >,
  <Axes: title={'center': 'Subsystem Z'}>,
  <Axes: ylabel='$\\text{pos}$ (m)'>,
  <Axes: >,
  <Axes: >],
 'plotter': <pyvista.plotting.plotter.Plotter at 0x2c4882b7770>,
 'scene_objects': {'rocket_actor': Actor (0x2c48976b040)
    Center:                     (0.32006999999999997, -0.0015085000000000237, 0.5884844999999999)
    Pickable:                   True
    Position:                   (0.0, 0.0, 0.0)
    Scale:                      (1.0, 1.0, 1.0)
    Visible:                    True
    X Bounds                    -6.402E-01, 1.280E+00
    Y Bounds                    -1.21

## Summary of Findings

### Key Insights from Testing:

1. **Hover Throttle**: ≈ 66.7% (from trim calculation)
   - This balances gravity perfectly when rocket is upright with no deflections

2. **Vertical Motion**:
   - **Ascend**: Pavg > hover throttle (e.g., 75%)
   - **Descend**: Pavg < hover throttle (e.g., 50%, but keep > 40% for safety)
   - Keep d1 = d2 = 0 to avoid tipping

3. **Rotation** (about body axes):
   - **About x-axis (roll)**: Use d1 servo deflection
   - **About y-axis (pitch)**: Use d2 servo deflection  
   - **About z-axis (yaw)**: Use Pdiff (differential throttle)

4. **Translation** (along world axes) - EMPIRICALLY DETERMINED:
   - **Along z**: Already covered by vertical motion
   - **Along +x**: Use **d2 < 0** (negative), increase Pavg to compensate
   - **Along +y**: Use **d1 > 0** (POSITIVE!), increase Pavg to compensate
   - **CRITICAL**: The actual signs differ from naive interpretation of equation (1)
   - Need higher throttle when tilted to maintain altitude

5. **Thrust Vector Direction** (from equation 1):
   ```
   b_eF = [sin(d2), -sin(d1)*cos(d2), cos(d1)*cos(d2)]
   ```
   **Empirically observed behavior (at upright orientation):**
   - d2 > 0 → rocket moves in **-x** direction
   - d2 < 0 → rocket moves in **+x** direction ✓
   - d1 > 0 → rocket moves in **+y** direction ✓
   - d1 < 0 → rocket moves in **-y** direction
   
   **To achieve positive world-frame motion:**
   - For +x: use **d2 < 0**
   - For +y: use **d1 > 0**

6. **Body Frame vs World Frame:**
   - There's a complex relationship between servo deflections and world-frame motion
   - The rotation matrices T_wb and the body frame orientation cause non-intuitive behavior
   - **Always test empirically** - don't trust naive sign assumptions!
   - Equation (1) gives thrust in **body frame**, which then transforms to world frame

### Understanding the Dynamics Implementation:

- `getForceAndMomentFromThrust()` correctly implements equations (1), (2), (3) ✓
- `f()` correctly implements the full dynamics equations (4), (5), (6) ✓
- The implementation matches the PDF description ✓
- **Sign convention**: Use the empirically tested values above for reliable control!